## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [3]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.flatten = layers.Flatten()
        self.W1 = tf.Variable(tf.random.normal([28*28, 128], stddev=0.1))
        self.b1 = tf.Variable(tf.zeros([128]))
        self.W2 = tf.Variable(tf.random.normal([128, 10], stddev=0.1))
        self.b2 = tf.Variable(tf.zeros([10]))

    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = self.flatten(x)
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [5]:
train_data, test_data = mnist_dataset()
for epoch in range(150):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.6362026 ; accuracy 0.09046666
epoch 1 : loss 2.6122506 ; accuracy 0.09018333
epoch 2 : loss 2.589737 ; accuracy 0.090566665
epoch 3 : loss 2.568497 ; accuracy 0.0911
epoch 4 : loss 2.5483906 ; accuracy 0.09193333
epoch 5 : loss 2.5292964 ; accuracy 0.093166664
epoch 6 : loss 2.5111098 ; accuracy 0.09443333
epoch 7 : loss 2.493741 ; accuracy 0.09601667
epoch 8 : loss 2.4771132 ; accuracy 0.0978
epoch 9 : loss 2.461157 ; accuracy 0.09991667
epoch 10 : loss 2.44581 ; accuracy 0.10261667
epoch 11 : loss 2.4310198 ; accuracy 0.10543333
epoch 12 : loss 2.41674 ; accuracy 0.1087
epoch 13 : loss 2.4029307 ; accuracy 0.11275
epoch 14 : loss 2.3895543 ; accuracy 0.11658333
epoch 15 : loss 2.376576 ; accuracy 0.120466664
epoch 16 : loss 2.3639677 ; accuracy 0.124283336
epoch 17 : loss 2.351704 ; accuracy 0.12858333
epoch 18 : loss 2.3397596 ; accuracy 0.13256666
epoch 19 : loss 2.328114 ; accuracy 0.13661666
epoch 20 : loss 2.316747 ; accuracy 0.14131667
epoch 21 : loss 2.3056393